This is not original at all!!
Data Munging: https://www.kaggle.com/mayer79/rnn-starter-for-huge-time-series (Please Upvote!)
LSTM: https://www.kaggle.com/thebrownviking20/intro-to-recurrent-neural-networks-lstm-gru (Please Upvote!)

In [1]:
# BASIC IDEA OF THE KERNEL

# The data consists of a one dimensional time series x with 600 Mio data points. 
# At test time, we will see a time series of length 150'000 to predict the next earthquake.
# The idea of this kernel is to randomly sample chunks of length 150'000 from x, derive some
# features and use them to update weights of a recurrent neural net with 150'000 / 1000 = 150
# time steps. 

import numpy as np 
import pandas as pd
import os
from tqdm import tqdm
# Define model
from keras.models import Sequential
from keras.layers import Dense, CuDNNGRU, Dropout, TimeDistributed, LSTM, CuDNNLSTM
from keras.optimizers import adam, RMSprop
from keras.callbacks import ModelCheckpoint
# Fix seeds
from numpy.random import seed
seed(639)
from tensorflow import set_random_seed
set_random_seed(5944)

from numpy.random import seed
seed(639)
from tensorflow import set_random_seed
set_random_seed(5944)

# Import
float_data = pd.read_csv("../input/train/train.csv", dtype={"acoustic_data": np.float32, "time_to_failure": np.float32}).values

# Helper function for the data generator. Extracts mean, standard deviation, and quantiles per time step.
# Can easily be extended. Expects a two dimensional array.
def extract_features(z):
     return np.c_[z.mean(axis=1), 
                  z.min(axis=1),
                  z.max(axis=1),
                  z.std(axis=1),
                  #np.median(z),
                  #np.transpose(np.percentile(np.abs(z), q=[0, 50, 75, 100], axis=1)),
                  np.sqrt(np.abs(z)).mean(axis=1),
                  np.sqrt(np.abs(z)).std(axis=1)
                 ]

# For a given ending position "last_index", we split the last 150'000 values 
# of "x" into 150 pieces of length 1000 each. So n_steps * step_length should equal 150'000.
# From each piece, a set features are extracted. This results in a feature matrix 
# of dimension (150 time steps x features).  
def create_X(x, last_index=None, n_steps=150, step_length=1000):
    if last_index == None:
        last_index=len(x)
       
    assert last_index - n_steps * step_length >= 0

    # Reshaping and approximate standardization with mean 5 and std 3.
    temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1) - 5 ) / 3
    
    # Extracts features of sequences of full length 1000, of the last 100 values and finally also 
    # of the last 10 observations. 
    return np.c_[extract_features(temp),
                 extract_features(temp[:, -step_length // 10:]),
                 extract_features(temp[:, -step_length // 100:]),
                 extract_features(temp[:, -step_length // 300:])]

# Query "create_X" to figure out the number of features
n_features = create_X(float_data[0:150000]).shape[1]
print("Our RNN is based on %i features"% n_features)
    
# The generator endlessly selects "batch_size" ending positions of sub-time series. For each ending position,
# the "time_to_failure" serves as target, while the features are created by the function "create_X".
def generator(data, min_index=0, max_index=None, batch_size=16, n_steps=150, step_length=1000):
    if max_index is None:
        max_index = len(data) - 1
     
    while True:
        # Pick indices of ending positions
        rows = np.random.randint(min_index + n_steps * step_length, max_index, size=batch_size)
         
        # Initialize feature matrices and targets
        samples = np.zeros((batch_size, n_steps, n_features))
        targets = np.zeros(batch_size, )
        
        for j, row in enumerate(rows):
            samples[j] = create_X(data[:, 0], last_index=row, n_steps=n_steps, step_length=step_length)
            targets[j] = data[row - 1, 1]
        yield samples, targets
        
batch_size = 32

# Position of second (of 16) earthquake. Used to have a clean split
# between train and validation
second_earthquake = 50085877
float_data[second_earthquake, 1]

# Initialize generators
train_gen = generator(float_data, batch_size=batch_size) # Use this for better score
# train_gen = generator(float_data, batch_size=batch_size, min_index=second_earthquake + 1)
valid_gen = generator(float_data, batch_size=batch_size, max_index=second_earthquake)

Using TensorFlow backend.


Our RNN is based on 24 features


In [ ]:
n_features

In [ ]:
%%time
cb = [ModelCheckpoint("model.hdf5", save_best_only=True, period=3)]

# The LSTM architecture
model = Sequential()
# First LSTM layer with Dropout regularisation
model.add(CuDNNLSTM(units=50, return_sequences=True, input_shape=(None,n_features)))
model.add(Dropout(0.2))
# Second LSTM layer
model.add(CuDNNLSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# Third LSTM layer
model.add(CuDNNLSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# Fourth LSTM layer
model.add(CuDNNLSTM(units=50))
model.add(Dropout(0.2))
# The output layer
model.add(Dense(units=1))

# Compiling the RNN


model.summary()

# Compile and fit model
model.compile(optimizer='rmsprop', loss='mae')



history = model.fit_generator(train_gen,
                              steps_per_epoch=1000,
                              epochs=40,
                              verbose=2,
                              callbacks=cb,
                              validation_data=valid_gen,
                              validation_steps=200
                             )





In [ ]:
# Visualize accuracies simply mean max min std 32 epoch
import matplotlib.pyplot as plt
%matplotlib inline
def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None
perf_plot(history) # Extra Layer

In [ ]:
# Visualize accuracies  simply mean max min std and sqrt mean 32 epoch
import matplotlib.pyplot as plt
%matplotlib inline
def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None
perf_plot(history) # Extra Layer

In [ ]:
# Visualize accuracies  simply mean max min std 40 epoch
import matplotlib.pyplot as plt
%matplotlib inline
def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None
perf_plot(history) # Extra Layer

In [ ]:
# Visualize accuracies simply mean max min std and sqrt mean 40 epoch
import matplotlib.pyplot as plt
%matplotlib inline
def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None
perf_plot(history) # Extra Layer

In [ ]:
# Visualize accuracies simply mean max min std and sqrt std 40 epoch
import matplotlib.pyplot as plt
%matplotlib inline
def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None
perf_plot(history) # Extra Layer

In [ ]:
# Visualize accuracies simply mean max min std and sqrt mean 80 epoch
import matplotlib.pyplot as plt
%matplotlib inline
def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None
perf_plot(history) # Extra Layer

In [ ]:
# Load submission file
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})
x = None
# Load each test data, create the feature matrix, get numeric prediction
for i, seg_id in enumerate(tqdm(submission.index)):
  #  print(i)
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    x = seg['acoustic_data'].values[:]
    #print(x.shape)
    submission.time_to_failure[i] = (model.predict(np.expand_dims(create_X(x), 0)))

# Save
submission.to_csv('submission-ltsm.csv')

In [ ]:
submission.head(10)

In [ ]:
train_features_df = pd.

In [ ]:
original features 
     return np.c_[z.mean(axis=1), 
                  z.min(axis=1),
                  z.max(axis=1),
                  z.std(axis=1)]
Epoch 1/32
 - 66s - loss: 2.3928 - val_loss: 1.9370
Epoch 2/32
 - 63s - loss: 2.2244 - val_loss: 1.7033
Epoch 3/32
 - 62s - loss: 2.1842 - val_loss: 1.9598
Epoch 4/32
 - 62s - loss: 2.1599 - val_loss: 1.9348
Epoch 5/32
 - 64s - loss: 2.1407 - val_loss: 1.7172
Epoch 6/32
 - 64s - loss: 2.1230 - val_loss: 1.7050
Epoch 7/32
 - 63s - loss: 2.1201 - val_loss: 1.9158
Epoch 8/32
 - 62s - loss: 2.1078 - val_loss: 1.7558
Epoch 9/32
 - 63s - loss: 2.1064 - val_loss: 1.7702
Epoch 10/32
 - 63s - loss: 2.1122 - val_loss: 1.6435
Epoch 11/32
 - 63s - loss: 2.0944 - val_loss: 1.6754
Epoch 12/32
 - 63s - loss: 2.0927 - val_loss: 1.6267
Epoch 13/32
 - 63s - loss: 2.0813 - val_loss: 1.7408
Epoch 14/32
 - 63s - loss: 2.0837 - val_loss: 1.8019
Epoch 15/32
 - 64s - loss: 2.0842 - val_loss: 1.6420
Epoch 16/32
 - 63s - loss: 2.0797 - val_loss: 1.7668
Epoch 17/32
 - 64s - loss: 2.0844 - val_loss: 1.7888
Epoch 18/32
 - 64s - loss: 2.0684 - val_loss: 1.7200
Epoch 19/32
 - 65s - loss: 2.0914 - val_loss: 1.8434
Epoch 20/32
 - 64s - loss: 2.0600 - val_loss: 1.9241
Epoch 21/32
 - 64s - loss: 2.0642 - val_loss: 1.7292
Epoch 22/32
 - 63s - loss: 2.0589 - val_loss: 1.8935
Epoch 23/32
 - 61s - loss: 2.0733 - val_loss: 1.7342
Epoch 24/32
 - 63s - loss: 2.0645 - val_loss: 1.5960
Epoch 25/32
 - 63s - loss: 2.0541 - val_loss: 1.7033
Epoch 26/32
 - 64s - loss: 2.0526 - val_loss: 1.6880
Epoch 27/32
 - 64s - loss: 2.0662 - val_loss: 1.7038
Epoch 28/32
 - 63s - loss: 2.0614 - val_loss: 1.7706
Epoch 29/32
 - 63s - loss: 2.0617 - val_loss: 1.6943
Epoch 30/32
 - 61s - loss: 2.0569 - val_loss: 1.7090
Epoch 31/32
 - 62s - loss: 2.0566 - val_loss: 1.6816
Epoch 32/32
 - 62s - loss: 2.0511 - val_loss: 1.6311


In [ ]:
     return np.c_[z.mean(axis=1), 
                  z.min(axis=1),
                  z.max(axis=1),
                  z.std(axis=1),
                  np.sqrt(np.abs(z)).mean(axis=1)

Epoch 1/32
 - 154s - loss: 2.3595 - val_loss: 1.9728
Epoch 2/32
 - 157s - loss: 2.2042 - val_loss: 1.9447
Epoch 3/32
 - 158s - loss: 2.1524 - val_loss: 1.7406
Epoch 4/32
 - 159s - loss: 2.1361 - val_loss: 1.5513
Epoch 5/32
 - 157s - loss: 2.1304 - val_loss: 1.5908
Epoch 6/32
 - 153s - loss: 2.1078 - val_loss: 1.6882
Epoch 7/32
 - 153s - loss: 2.0924 - val_loss: 1.6414
Epoch 8/32
 - 152s - loss: 2.0977 - val_loss: 1.6917
Epoch 9/32
 - 152s - loss: 2.0910 - val_loss: 1.9564
Epoch 10/32
 - 152s - loss: 2.0816 - val_loss: 1.7140
Epoch 11/32
 - 153s - loss: 2.0708 - val_loss: 1.7071
Epoch 12/32
 - 154s - loss: 2.0761 - val_loss: 1.8837
Epoch 13/32
 - 159s - loss: 2.0632 - val_loss: 1.7577
Epoch 14/32
 - 157s - loss: 2.0771 - val_loss: 1.6824
Epoch 15/32
 - 158s - loss: 2.0725 - val_loss: 1.6208
Epoch 16/32
 - 158s - loss: 2.0668 - val_loss: 1.5576
Epoch 17/32
 - 157s - loss: 2.0586 - val_loss: 1.8591
Epoch 18/32
 - 158s - loss: 2.0547 - val_loss: 1.8409
Epoch 19/32
 - 154s - loss: 2.0550 - val_loss: 1.4596
Epoch 20/32
 - 157s - loss: 2.0596 - val_loss: 1.5710
Epoch 21/32
 - 152s - loss: 2.0535 - val_loss: 1.8322
Epoch 22/32
 - 152s - loss: 2.0464 - val_loss: 1.7149
Epoch 23/32
 - 156s - loss: 2.0430 - val_loss: 1.5419
Epoch 24/32
 - 155s - loss: 2.0358 - val_loss: 1.6811
Epoch 25/32
 - 157s - loss: 2.0327 - val_loss: 1.7708
Epoch 26/32
 - 158s - loss: 2.0374 - val_loss: 1.7572
Epoch 27/32
 - 153s - loss: 2.0326 - val_loss: 1.5652
Epoch 28/32
 - 153s - loss: 2.0308 - val_loss: 1.5978
Epoch 29/32
 - 155s - loss: 2.0450 - val_loss: 1.7402
Epoch 30/32
 - 156s - loss: 2.0325 - val_loss: 1.7172
Epoch 31/32
 - 154s - loss: 2.0342 - val_loss: 1.6428
Epoch 32/32
 - 154s - loss: 2.0261 - val_loss: 1.6561
CPU times: user 1h 49min 6s, sys: 3min, total: 1h 52min 7s
Wall time: 1h 22min 49s


In [ ]:
     return np.c_[z.mean(axis=1), 
                  z.min(axis=1),
                  z.max(axis=1),
                  z.std(axis=1),
                  #np.sqrt(np.abs(z)).mean(axis=1)
Epoch 1/40
 - 65s - loss: 2.3928 - val_loss: 1.9375
Epoch 2/40
 - 64s - loss: 2.2232 - val_loss: 1.6909
Epoch 3/40
 - 64s - loss: 2.1922 - val_loss: 2.1060
Epoch 4/40
 - 64s - loss: 2.1631 - val_loss: 1.9303
Epoch 5/40
 - 64s - loss: 2.1422 - val_loss: 1.7295
Epoch 6/40
 - 64s - loss: 2.1202 - val_loss: 1.6751
Epoch 7/40
 - 64s - loss: 2.1185 - val_loss: 1.7106
Epoch 8/40
 - 64s - loss: 2.1116 - val_loss: 1.9057
Epoch 9/40
 - 64s - loss: 2.1101 - val_loss: 1.5614
Epoch 10/40
 - 64s - loss: 2.1172 - val_loss: 1.6790
Epoch 11/40
 - 64s - loss: 2.0888 - val_loss: 1.6710
Epoch 12/40
 - 65s - loss: 2.0919 - val_loss: 1.5902
Epoch 13/40
 - 64s - loss: 2.0823 - val_loss: 1.6984
Epoch 14/40
 - 65s - loss: 2.0856 - val_loss: 1.7742
Epoch 15/40
 - 64s - loss: 2.0857 - val_loss: 1.7923
Epoch 16/40
 - 64s - loss: 2.0792 - val_loss: 1.7989
Epoch 17/40
 - 63s - loss: 2.0865 - val_loss: 1.8651
Epoch 18/40
 - 62s - loss: 2.0748 - val_loss: 1.7109
Epoch 19/40
 - 63s - loss: 2.0905 - val_loss: 1.7494
Epoch 20/40
 - 63s - loss: 2.0589 - val_loss: 1.7050
Epoch 21/40
 - 62s - loss: 2.0642 - val_loss: 1.5661
Epoch 22/40
 - 62s - loss: 2.0519 - val_loss: 1.7448
Epoch 23/40
 - 62s - loss: 2.0674 - val_loss: 1.7525
Epoch 24/40
 - 62s - loss: 2.0720 - val_loss: 1.6188
Epoch 25/40
 - 62s - loss: 2.0571 - val_loss: 1.8109
Epoch 26/40
 - 61s - loss: 2.0515 - val_loss: 1.6783
Epoch 27/40
 - 62s - loss: 2.0624 - val_loss: 1.8582
Epoch 28/40
 - 63s - loss: 2.0618 - val_loss: 1.7183
Epoch 29/40
 - 64s - loss: 2.0642 - val_loss: 1.7743
Epoch 30/40
 - 63s - loss: 2.0533 - val_loss: 1.7152
Epoch 31/40
 - 62s - loss: 2.0643 - val_loss: 1.8029
Epoch 32/40
 - 62s - loss: 2.0507 - val_loss: 1.5906
Epoch 33/40
 - 62s - loss: 2.0491 - val_loss: 1.7614
Epoch 34/40
 - 62s - loss: 2.0578 - val_loss: 1.5521
Epoch 35/40
 - 61s - loss: 2.0408 - val_loss: 1.7914
Epoch 36/40
 - 64s - loss: 2.0512 - val_loss: 1.7751
Epoch 37/40
 - 63s - loss: 2.0403 - val_loss: 1.6504
Epoch 38/40
 - 64s - loss: 2.0427 - val_loss: 1.8024
Epoch 39/40
 - 63s - loss: 2.0462 - val_loss: 1.6763
Epoch 40/40
 - 63s - loss: 2.0490 - val_loss: 1.6260
CPU times: user 1h 11min 48s, sys: 3min 23s, total: 1h 15min 11s
Wall time: 42min 10s

In [ ]:
     return np.c_[z.mean(axis=1), 
                  z.min(axis=1),
                  z.max(axis=1),
                  z.std(axis=1),
                  np.sqrt(np.abs(z)).mean(axis=1)

Epoch 1/40
 - 81s - loss: 2.3920 - val_loss: 2.0196
Epoch 2/40
 - 80s - loss: 2.2248 - val_loss: 1.7401
Epoch 3/40
 - 78s - loss: 2.1981 - val_loss: 1.9153
Epoch 4/40
 - 78s - loss: 2.1820 - val_loss: 1.8003
Epoch 5/40
 - 79s - loss: 2.1465 - val_loss: 1.6427
Epoch 6/40
 - 78s - loss: 2.1354 - val_loss: 1.7191
Epoch 7/40
 - 78s - loss: 2.1312 - val_loss: 1.8521
Epoch 8/40
 - 78s - loss: 2.1288 - val_loss: 1.6283
Epoch 9/40
 - 78s - loss: 2.1173 - val_loss: 1.7793
Epoch 10/40
 - 78s - loss: 2.1200 - val_loss: 1.7659
Epoch 11/40
 - 82s - loss: 2.1033 - val_loss: 1.6827
Epoch 12/40
 - 81s - loss: 2.0991 - val_loss: 1.7370
Epoch 13/40
 - 80s - loss: 2.0918 - val_loss: 1.7541
Epoch 14/40
 - 80s - loss: 2.0945 - val_loss: 1.7652
Epoch 15/40
 - 82s - loss: 2.0921 - val_loss: 1.5560
Epoch 16/40
 - 82s - loss: 2.0881 - val_loss: 1.7586
Epoch 17/40
 - 82s - loss: 2.0883 - val_loss: 1.6854
Epoch 18/40
 - 82s - loss: 2.0814 - val_loss: 1.7114
Epoch 19/40
 - 82s - loss: 2.0999 - val_loss: 1.8670
Epoch 20/40
 - 82s - loss: 2.0672 - val_loss: 1.7266
Epoch 21/40
 - 82s - loss: 2.0704 - val_loss: 1.8502
Epoch 22/40
 - 82s - loss: 2.0609 - val_loss: 2.0490
Epoch 23/40
 - 82s - loss: 2.0799 - val_loss: 1.7238
Epoch 24/40
 - 82s - loss: 2.0702 - val_loss: 1.8085
Epoch 25/40
 - 82s - loss: 2.0608 - val_loss: 1.6307
Epoch 26/40
 - 82s - loss: 2.0554 - val_loss: 1.7432
Epoch 27/40
 - 82s - loss: 2.0680 - val_loss: 1.7395
Epoch 28/40
 - 82s - loss: 2.0645 - val_loss: 1.7326
Epoch 29/40
 - 83s - loss: 2.0655 - val_loss: 1.8699
Epoch 30/40
 - 83s - loss: 2.0550 - val_loss: 1.7695
Epoch 31/40
 - 82s - loss: 2.0706 - val_loss: 1.7876
Epoch 32/40
 - 82s - loss: 2.0552 - val_loss: 1.5393
Epoch 33/40
 - 83s - loss: 2.0514 - val_loss: 1.7154
Epoch 34/40
 - 82s - loss: 2.0528 - val_loss: 1.5252
Epoch 35/40
 - 82s - loss: 2.0446 - val_loss: 1.7102
Epoch 36/40
 - 82s - loss: 2.0560 - val_loss: 1.6044
Epoch 37/40
 - 82s - loss: 2.0458 - val_loss: 1.7797
Epoch 38/40
 - 82s - loss: 2.0429 - val_loss: 1.8065
Epoch 39/40
 - 82s - loss: 2.0535 - val_loss: 1.6717
Epoch 40/40
 - 82s - loss: 2.0459 - val_loss: 1.5742
CPU times: user 1h 26min 2s, sys: 3min 31s, total: 1h 29min 33s
Wall time: 54min 10s

In [ ]:
np.c_[z.mean(axis=1), 
                  z.min(axis=1),
                  z.max(axis=1),
                  z.std(axis=1),
                  #np.sqrt(np.abs(z)).mean(axis=1),
                  np.sqrt(np.abs(z)).std(axis=1)
                 ]
Epoch 1/40
 - 93s - loss: 2.4005 - val_loss: 1.9946
Epoch 2/40
 - 93s - loss: 2.2334 - val_loss: 1.8570
Epoch 3/40
 - 92s - loss: 2.2080 - val_loss: 1.7356
Epoch 4/40
 - 93s - loss: 2.1833 - val_loss: 1.7988
Epoch 5/40
 - 92s - loss: 2.1430 - val_loss: 1.6780
Epoch 6/40
 - 92s - loss: 2.1329 - val_loss: 1.6404
Epoch 7/40
 - 91s - loss: 2.1278 - val_loss: 1.8292
Epoch 8/40
 - 92s - loss: 2.1261 - val_loss: 1.6718
Epoch 9/40
 - 92s - loss: 2.1168 - val_loss: 1.8089
Epoch 10/40
 - 93s - loss: 2.1255 - val_loss: 1.8600
Epoch 11/40
 - 93s - loss: 2.0972 - val_loss: 1.6522
Epoch 12/40
 - 92s - loss: 2.0992 - val_loss: 1.6430
Epoch 13/40
 - 92s - loss: 2.0946 - val_loss: 1.7748
Epoch 14/40
 - 92s - loss: 2.0960 - val_loss: 1.7877
Epoch 15/40
 - 93s - loss: 2.0943 - val_loss: 1.7881
Epoch 16/40
 - 92s - loss: 2.0889 - val_loss: 1.7832
Epoch 17/40
 - 92s - loss: 2.0959 - val_loss: 1.7639
Epoch 18/40
 - 93s - loss: 2.0829 - val_loss: 1.6759
Epoch 19/40
 - 92s - loss: 2.1061 - val_loss: 1.7114
Epoch 20/40
 - 94s - loss: 2.0677 - val_loss: 1.7818
Epoch 21/40
 - 95s - loss: 2.0712 - val_loss: 1.6394
Epoch 22/40
 - 93s - loss: 2.0623 - val_loss: 1.7075
Epoch 23/40
 - 93s - loss: 2.0761 - val_loss: 1.8154
Epoch 24/40
 - 93s - loss: 2.0747 - val_loss: 1.5792
Epoch 25/40
 - 93s - loss: 2.0627 - val_loss: 1.7405
Epoch 26/40
 - 94s - loss: 2.0548 - val_loss: 1.6842
Epoch 27/40
 - 95s - loss: 2.0660 - val_loss: 1.8134
Epoch 28/40
 - 95s - loss: 2.0645 - val_loss: 1.7107
Epoch 29/40
 - 94s - loss: 2.0645 - val_loss: 1.7149
Epoch 30/40
 - 93s - loss: 2.0582 - val_loss: 1.6315
Epoch 31/40
 - 92s - loss: 2.0634 - val_loss: 1.7321
Epoch 32/40
 - 92s - loss: 2.0578 - val_loss: 1.6383
Epoch 33/40
 - 93s - loss: 2.0539 - val_loss: 1.7767
Epoch 34/40
 - 91s - loss: 2.0546 - val_loss: 1.5094
Epoch 35/40
 - 94s - loss: 2.0388 - val_loss: 1.7988
Epoch 36/40
 - 95s - loss: 2.0541 - val_loss: 1.7876
Epoch 37/40
 - 93s - loss: 2.0431 - val_loss: 1.6930
Epoch 38/40
 - 95s - loss: 2.0412 - val_loss: 1.8164
Epoch 39/40
 - 92s - loss: 2.0475 - val_loss: 1.6933
Epoch 40/40
 - 93s - loss: 2.0496 - val_loss: 1.6468
CPU times: user 1h 33min 4s, sys: 3min 27s, total: 1h 36min 31s
Wall time: 1h 1min 59s

In [ ]:
np.c_[z.mean(axis=1), 
                  z.min(axis=1),
                  z.max(axis=1),
                  z.std(axis=1),
                  #np.median(z),
                  np.transpose(np.percentile(np.abs(z), q=[0, 50, 75, 100], axis=1)),
                  np.sqrt(np.abs(z)).mean(axis=1)
                  #np.sqrt(np.abs(z)).std(axis=1)
Epoch 1/80
 - 275s - loss: 2.4420 - val_loss: 2.0530
Epoch 2/80
 - 271s - loss: 2.2485 - val_loss: 1.6975
Epoch 3/80
 - 274s - loss: 2.2377 - val_loss: 1.8995
Epoch 4/80
 - 280s - loss: 2.2032 - val_loss: 1.9955
Epoch 5/80
 - 272s - loss: 2.1795 - val_loss: 1.7917
Epoch 6/80
 - 272s - loss: 2.1598 - val_loss: 1.9248
Epoch 7/80
 - 273s - loss: 2.1371 - val_loss: 1.8859
Epoch 8/80
 - 271s - loss: 2.1348 - val_loss: 1.7393
Epoch 9/80
 - 274s - loss: 2.1232 - val_loss: 1.7991
Epoch 10/80
 - 272s - loss: 2.1259 - val_loss: 1.8371
Epoch 11/80
 - 270s - loss: 2.1127 - val_loss: 1.6605
Epoch 12/80
 - 270s - loss: 2.1033 - val_loss: 1.7167
Epoch 13/80
 - 270s - loss: 2.0987 - val_loss: 1.7000
Epoch 14/80
 - 271s - loss: 2.0965 - val_loss: 1.8395
Epoch 15/80
 - 270s - loss: 2.0941 - val_loss: 1.6249
Epoch 16/80
 - 270s - loss: 2.0866 - val_loss: 1.7929
Epoch 17/80
 - 269s - loss: 2.0915 - val_loss: 1.7493
Epoch 18/80
 - 271s - loss: 2.0837 - val_loss: 1.6512
Epoch 19/80
 - 272s - loss: 2.0933 - val_loss: 1.8019
Epoch 20/80
 - 280s - loss: 2.0643 - val_loss: 1.7801
Epoch 21/80
 - 275s - loss: 2.0724 - val_loss: 1.7586
Epoch 22/80
 - 277s - loss: 2.0600 - val_loss: 1.8950
Epoch 23/80
 - 278s - loss: 2.0789 - val_loss: 1.7047
Epoch 24/80
 - 278s - loss: 2.0738 - val_loss: 1.6130
Epoch 25/80
 - 278s - loss: 2.0614 - val_loss: 1.7254
Epoch 26/80
 - 279s - loss: 2.0568 - val_loss: 1.6731
Epoch 27/80
 - 273s - loss: 2.0701 - val_loss: 1.6532
Epoch 28/80
 - 274s - loss: 2.0681 - val_loss: 1.8125
Epoch 29/80
 - 279s - loss: 2.0643 - val_loss: 1.7123
Epoch 30/80
 - 271s - loss: 2.0588 - val_loss: 1.7218
Epoch 31/80
 - 272s - loss: 2.0610 - val_loss: 1.6583
Epoch 32/80
 - 273s - loss: 2.0515 - val_loss: 1.5537
Epoch 33/80
 - 274s - loss: 2.0492 - val_loss: 1.7274
Epoch 34/80
 - 274s - loss: 2.0550 - val_loss: 1.5911
Epoch 35/80
 - 273s - loss: 2.0342 - val_loss: 1.6914
Epoch 36/80
 - 278s - loss: 2.0532 - val_loss: 1.6069
Epoch 37/80
 - 277s - loss: 2.0480 - val_loss: 1.6758
Epoch 38/80
 - 281s - loss: 2.0436 - val_loss: 1.6805
Epoch 39/80
 - 274s - loss: 2.0514 - val_loss: 1.7445
Epoch 40/80
 - 278s - loss: 2.0446 - val_loss: 1.5711
Epoch 41/80
 - 269s - loss: 2.0459 - val_loss: 1.7405
Epoch 42/80
 - 271s - loss: 2.0530 - val_loss: 1.6143
Epoch 43/80
 - 268s - loss: 2.0355 - val_loss: 1.6658
Epoch 44/80
 - 270s - loss: 2.0475 - val_loss: 1.7299
Epoch 45/80
 - 273s - loss: 2.0311 - val_loss: 1.6779
Epoch 46/80
 - 276s - loss: 2.0267 - val_loss: 1.5801
Epoch 47/80
 - 270s - loss: 2.0130 - val_loss: 1.6804
Epoch 48/80
 - 271s - loss: 2.0446 - val_loss: 1.6024
Epoch 49/80
 - 274s - loss: 2.0306 - val_loss: 1.7541
Epoch 50/80
 - 279s - loss: 2.0236 - val_loss: 1.7320
Epoch 51/80
 - 278s - loss: 2.0386 - val_loss: 1.5982
Epoch 52/80
 - 279s - loss: 2.0289 - val_loss: 1.6217
Epoch 53/80
 - 279s - loss: 2.0206 - val_loss: 1.8180
Epoch 54/80
 - 276s - loss: 2.0273 - val_loss: 1.6369
Epoch 55/80
 - 272s - loss: 2.0260 - val_loss: 1.6120
Epoch 56/80
 - 272s - loss: 2.0200 - val_loss: 1.6388
Epoch 57/80
 - 271s - loss: 2.0403 - val_loss: 1.6474
Epoch 58/80
 - 275s - loss: 2.0137 - val_loss: 1.7147
Epoch 59/80
 - 269s - loss: 2.0213 - val_loss: 1.6805
Epoch 60/80
 - 268s - loss: 2.0259 - val_loss: 1.6119
Epoch 61/80
 - 271s - loss: 2.0210 - val_loss: 1.4685
Epoch 62/80
 - 271s - loss: 2.0194 - val_loss: 1.6699
Epoch 63/80
 - 268s - loss: 2.0120 - val_loss: 1.6508
Epoch 64/80
 - 269s - loss: 2.0140 - val_loss: 1.7159
Epoch 65/80
 - 268s - loss: 2.0157 - val_loss: 1.8277
Epoch 66/80
 - 274s - loss: 2.0018 - val_loss: 1.6347
Epoch 67/80
 - 268s - loss: 2.0126 - val_loss: 1.7237
Epoch 68/80
 - 269s - loss: 1.9904 - val_loss: 1.6448
Epoch 69/80
 - 270s - loss: 1.9926 - val_loss: 1.6572
Epoch 70/80
 - 268s - loss: 2.0086 - val_loss: 1.5817
Epoch 71/80
 - 271s - loss: 1.9957 - val_loss: 1.6470
Epoch 72/80
 - 270s - loss: 1.9928 - val_loss: 1.6598
Epoch 73/80
 - 269s - loss: 2.0029 - val_loss: 1.7102
Epoch 74/80
 - 272s - loss: 1.9949 - val_loss: 1.6547
Epoch 75/80
 - 270s - loss: 1.9831 - val_loss: 1.5767
Epoch 76/80
 - 269s - loss: 1.9996 - val_loss: 1.7222
Epoch 77/80
 - 275s - loss: 1.9796 - val_loss: 1.5692
Epoch 78/80
 - 279s - loss: 1.9908 - val_loss: 1.6256
Epoch 79/80
 - 279s - loss: 1.9838 - val_loss: 1.6479
Epoch 80/80
 - 279s - loss: 1.9672 - val_loss: 1.6011
CPU times: user 7h 11min 50s, sys: 7min 10s, total: 7h 19min
Wall time: 6h 4min 18s

# Visualiz